In [27]:
import numpy as np

import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import Angle, EarthLocation, AltAz, SkyCoord
from astropy.time import Time

In [50]:
MMT_LOCATION = EarthLocation.from_geodetic("-110:53:04.4", "31:41:19.6", 2600 * u.m)
obstime = Time("2021-08-21T06:00:00", format='isot')

# don't strictly need time or location for our purposes, but astropy wants them for defining the alt/az frame
aa_frame = AltAz(obstime=obstime, location=MMT_LOCATION)

def tpoint(coo, ia, ie, an, aw, ca, npae, tf, tx):
    da = -1 * ia
    da -= an * np.sin(coo.az) * np.tan(coo.alt)
    da -= aw * np.cos(coo.az) * np.tan(coo.alt)
    da -= ca / np.cos(coo.alt)
    da -= npae * np.tan(coo.alt)

    de = ie
    de -= an * np.cos(coo.az)
    de += aw * np.sin(coo.az)
    de -= tf * np.cos(coo.alt)
    de -= tx / np.tan(coo.alt)

    new_az = coo.az + da * u.arcsec
    new_alt = coo.alt + de * u.arcsec

    new_coo = SkyCoord(new_az, new_alt, frame=aa_frame)
    return new_coo

In [30]:
datafile = "point_tweak.dat"
t = ascii.read(datafile, data_start=20, format='no_header', guess=False, fast_reader=False)
az_obs = Angle(t['col1'], unit=u.degree).wrap_at(360 * u.deg)
el_obs = Angle(t['col2'], unit=u.degree).wrap_at(360 * u.deg)
az_raw = Angle(t['col3'], unit=u.degree).wrap_at(360 * u.deg)
el_raw = Angle(t['col4'], unit=u.degree).wrap_at(360 * u.deg)
coo_obs = SkyCoord(az_obs, el_obs, frame=aa_frame)
coo_raw = SkyCoord(az_raw, el_raw, frame=aa_frame)

In [42]:
coo_obs.separation(coo_raw).std().to(u.arcsec)

<Angle 321.7543166 arcsec>

In [51]:
t_coo = tpoint(coo_raw, 1212.01, -2.99, 2.51, -10.34, -10.10, 0.0, 21.41, -2.71)
coo_obs.separation(t_coo).std().to(u.arcsec)

<Angle 0.46492157 arcsec>

In [23]:
az_obs - az_raw

<Angle [-0.3360808, -0.3359315, -0.33442  , -0.3346809, -0.3347286,
        -0.3349824, -0.3359607, -0.3372572, -0.3351022, -0.3355928,
        -0.3354866, -0.3345204, -0.3341706, -0.331776 , -0.3330189,
        -0.3315314, -0.3286501, -0.3318254, -0.3298696, -0.3230378,
        -0.3323679, -0.3290886, -0.32377  , -0.3331476, -0.3286387,
        -0.3218772, -0.3330954, -0.3311469, -0.3248421, -0.3335499,
        -0.3321328, -0.3302185, -0.3338395, -0.3335516, -0.3317048,
        -0.3342735, -0.3342565, -0.3327243, -0.3340663, -0.3337756,
        -0.3325001, -0.3345057, -0.3344697, -0.3327072, -0.3366478,
        -0.3339039, -0.3336627, -0.3314356, -0.3331838, -0.3310763,
        -0.3291577, -0.3319177, -0.3295819, -0.3231104, -0.3324353,
        -0.3291824, -0.3243064, -0.3328227, -0.3303804, -0.3242628,
        -0.3320903, -0.3310985, -0.3278456, -0.3336246, -0.3317314,
        -0.3298435, -0.3172888, -0.3027352, -0.2796584, -0.2934557,
        -0.3160199, -0.319441 , -0.3207713, -0.3